# 

In [23]:
#!/usr/bin/env python
from collections import OrderedDict
import matplotlib.pyplot as plt
from itertools import groupby
from itertools import islice
import pandas as pd
import numpy as np
import collections
import itertools
import operator
import natsort
import math
import csv
import os

######### INTRACELLULAR MUTUAL INFORMATION CALCULATION #############

# load the files MS_7bits.csv or BT_7bits.csv with AllFBAs_7.csv media files with correct local spath. BT = B.theta MS=M.smithii
BT_MIscores_df = pd.read_csv('Mutual_Information_Analysis_Module/BT_7bits.csv')
MS_MIscores_df = pd.read_csv('Mutual_Information_Analysis_Module/MS_7bits.csv')
All_scores_ary = pd.read_csv('Mutual_Information_Analysis_Module/AllFBAs_7.csv') #.to_numpy()[:,1:]

# if ((BT_MIscores_df[1]-1 != All_scores_ary.shape[0]) or (MS_MIscores_df[1]-1 != All_scores_ary.shape[0]) or 
#     not np.array_equal(All_scores_ary, All_scores_ary.astype(bool)) or (int(math.log(s_df2[0],2)) != (All_scores_ary.shape[1])-1)
#    ):
#     print ('invalid input values')

#-----All possible combination of the chemical compounds----------------------
# 2.0 Sperating m0 from rest of the lables


# TODO - what is the purpose of this loop?
for col in All_scores_ary.columns:
    All_scores_ary.loc[All_scores_ary[col] == 1 , col] = col

# 2.1 Creating a disctionary for all FBAs except m0
mydict = {'m0': "['0']"}
for x in range(0, len(All_scores_ary)):
    for i in range(1, All_scores_ary.shape[1]):
        currentvalue = All_scores_ary.iloc[x,i]
        currentID = All_scores_ary.iloc[x, 0]
        if currentID == "m0":  continue
        mydict.setdefault(currentID,[])
        mydict[currentID].append(currentvalue)
mydict = collections.OrderedDict(natsort.natsorted(mydict.items()))

# List of Compounds combination in the list
Compounds_Combi = list(range(1, All_scores_ary.shape[1]+1))
my_combi_list = [list(subset) for L in range(0, len(Compounds_Combi)+1)
                 for subset in itertools.combinations(Compounds_Combi, L)]
print("consumed combinations", my_combi_list)

# Created a dictionary where the keys:
# list of compounds combination
# values are corresponding FBAs list in df2
result_dict = {}
for element in my_combi_list[1:]:
    for k, v in mydict.items():
        if set(v).issubset(set(map(str, element))):    # TODO - what does this mean?
            key = ','.join(map(str, element))
            if key in result_dict:  result_dict[key].add(k)
            else:  print(key, k) ; result_dict.update({key: set(["m0", k])})
print(result_dict)

# Created a dictionary where the keys are:
# list of compounds combination
# values are compounds combination FBAs with df1 vaules
df_BT = BT_MIscores_df[list(result_dict.values())[0]].drop_duplicates(keep="first")
df_MS = MS_MIscores_df[list(result_dict.values())[0]].drop_duplicates(keep="first")
MI_dict = {}
for df in [df_BT, df_MS]:
    for index, row in df.iterrows():   # row
        for col in range(len(cols)-1):   # column
            duplicated = []
            v = row[col]
            
            # TODO stopped here
            for j in range(i+1,len(cols)):
                if np.array_equal(v, df[k][cols[j]].to_numpy()):
                    remove.append(cols[j])
                    duplicated.append(cols[j])
            if duplicated and cols[i] not in remove:
                removed.update({cols[i]:duplicated})
            count = {key: len(value) for key, value in removed.items()}
            values = map(lambda x: x+1, count.to_numpy())
            d = {x:values.count(x) for x in values}

        #-------Mutual Inforamtion (MI) calculation-------------
        FBAs = len(df[1][k].columns)
        uncertainty = math.log(FBAs,2)
        conditional_uncertainty = math.log(1, 2) if not values else sum([-d[key] * (key/float(FBAs)) * math.log(1.0/key,2) for key in d])
        MI_dict.update({df[0][k] : uncertainty - conditional_uncertainty})

#Sorted MI_dict
MI_dict = OrderedDict(sorted(MI_dict.items(), key=lambda x: (len(x[0]),x[1]), reverse=True))
print(MI_dict)

# Plot MI_dict
plt.bar(range(len(MI_dict)), MI_dict.values(), align='center', alpha=0.5, width=0.7)
plt.xticks(range(len(MI_dict)), MI_dict.keys(), rotation='vertical')
plt.xlabel('Compund Combinations')
plt.ylabel('Mutual Information (in Bits)')
plt.title("Organism:XYZ")
fig1 = plt.gcf()
plt.show()
# plt.draw()
# fig1.savefig('MI_plot.png', dpi=100)

#import pdb; pdb.set_trace()

consumed combinations [[], [1], [2], [3], [4], [5], [6], [7], [8], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [4, 5], [4, 6], [4, 7], [4, 8], [5, 6], [5, 7], [5, 8], [6, 7], [6, 8], [7, 8], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 2, 8], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 3, 8], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 4, 8], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 6, 7], [1, 6, 8], [1, 7, 8], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7], [2, 3, 8], [2, 4, 5], [2, 4, 6], [2, 4, 7], [2, 4, 8], [2, 5, 6], [2, 5, 7], [2, 5, 8], [2, 6, 7], [2, 6, 8], [2, 7, 8], [3, 4, 5], [3, 4, 6], [3, 4, 7], [3, 4, 8], [3, 5, 6], [3, 5, 7], [3, 5, 8], [3, 6, 7], [3, 6, 8], [3, 7, 8], [4, 5, 6], [4, 5, 7], [4, 5, 8], [4, 6, 7], [4, 6, 8], [4, 7, 8], [5, 6, 7], [5, 6, 8], [5, 7, 8], [6, 7, 8], [1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 3, 6], [1, 2, 3, 7], [1, 2, 3, 8], [1, 2, 4, 5], [1, 2, 4,

/var/folders/rp/9cl8flv169g9fq27kxq24xjh0000gs/T/ipykernel_41295/3780673780.py:68: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_BT = BT_MIscores_df[list(result_dict.values())[0]].drop_duplicates(keep="first")
/var/folders/rp/9cl8flv169g9fq27kxq24xjh0000gs/T/ipykernel_41295/3780673780.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_MS = MS_MIscores_df[list(result_dict.values())[0]].drop_duplicates(keep="first")


NameError: name 'cols' is not defined